In [ ]:
Questo notebook utilizza un modello CUT pre-addestrato per trasformare le immagini del dominio **Reale** nello stile del dominio **Sintetico**.

**Prerequisiti:**
1. Avere il Dataset delle immagini caricato (es. `EGO-CH-OBJ-SEG`).
2. Avere il Dataset dei pesi caricato (il file `.pth` ottenuto dal training).

In [ ]:
import os
import shutil


%cd /kaggle/working


repo_name = "benchmarking-generative-models-for-domain-adaptation"
if os.path.exists(repo_name):
    shutil.rmtree(repo_name)


print("Clono la repository...")
!git clone --recursive https://github.com/AntonioRosano/benchmarking-generative-models-for-domain-adaptation.git
%cd {repo_name}

print("Installo librerie...")
!pip install -q dominate visdom

In [ ]:
# Patch per il bug noto di argparse in cut_model.py
target_file = "/kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/cut/models/cut_model.py"

if os.path.exists(target_file):
    with open(target_file, "r") as f:
        content = f.read()
    
    new_content = content.replace("choices='(CUT, cut, FastCUT, fastcut)'", "choices=['CUT', 'cut', 'FastCUT', 'fastcut']")
    
    with open(target_file, "w") as f:
        f.write(new_content)
    print("Patch applicata correttamente.")
else:
    print("Errore: File cut_model.py non trovato.")

In [ ]:
import glob

# --- DEFINIZIONE PERCORSI ---
KAGGLE_REAL_IMAGES = "/kaggle/input/datasets/antoniorosano/ego-ch/EGO-CH-OBJ-SEG/real/train/frames"
CUT_WEIGHTS_SOURCE = "/kaggle/input/datasets/antoniorosano/cut-weights/CUT_logs/ego_cut_kaggle/latest_net_G.pth"

REPO_DIR = "/kaggle/working/benchmarking-generative-models-for-domain-adaptation"
CUT_DIR = os.path.join(REPO_DIR, "models/cut")
DATASET_DIR = os.path.join(CUT_DIR, "datasets/ego_adaptation")


TEST_A_DIR = os.path.join(DATASET_DIR, "testA") 
TEST_B_DIR = os.path.join(DATASET_DIR, "testB")
TRAIN_A_DIR = os.path.join(DATASET_DIR, "trainA") 
TRAIN_B_DIR = os.path.join(DATASET_DIR, "trainB") 

CHECKPOINT_DIR = os.path.join(CUT_DIR, "checkpoints/ego_cut_kaggle")
RESULTS_DIR = "/kaggle/working/real_stylized_as_synthetic"

# --- PATCH CUT ---
target_file = os.path.join(CUT_DIR, "models/cut_model.py")
if os.path.exists(target_file):
    with open(target_file, "r") as f: content = f.read()
    new_content = content.replace("choices='(CUT, cut, FastCUT, fastcut)'", "choices=['CUT', 'cut', 'FastCUT', 'fastcut']")
    with open(target_file, "w") as f: f.write(new_content)
    print("Patch applicata.")

# --- PREPARAZIONE DATASET ---
%cd {CUT_DIR}

# Prepariamo testA
if os.path.exists(TEST_A_DIR): shutil.rmtree(TEST_A_DIR)
os.makedirs(TEST_A_DIR, exist_ok=True)

print(f"Copia immagini da {KAGGLE_REAL_IMAGES} a {TEST_A_DIR}...")
image_list = glob.glob(os.path.join(KAGGLE_REAL_IMAGES, "*"))
for img_path in image_list:
    shutil.copy(img_path, TEST_A_DIR)
    
num_img = len(os.listdir(TEST_A_DIR))
print(f"Copiate {num_img} immagini in testA.")

# Prepariamo le cartelle DUMMY (trainA, trainB, testB)
dummy_dirs = [TEST_B_DIR, TRAIN_A_DIR, TRAIN_B_DIR]

for d in dummy_dirs:
    if os.path.exists(d): shutil.rmtree(d)
    os.makedirs(d, exist_ok=True)
    # Copiamo UNA immagine a caso dentro
    if num_img > 0:
        shutil.copy(image_list[0], d)



if os.path.exists(CHECKPOINT_DIR): shutil.rmtree(CHECKPOINT_DIR)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

dest_weights = os.path.join(CHECKPOINT_DIR, "latest_net_G.pth")
if os.path.exists(CUT_WEIGHTS_SOURCE):
    shutil.copy(CUT_WEIGHTS_SOURCE, dest_weights)
    print(f"Pesi copiati in: {dest_weights}")
else:
    print(f"ERRORE: Non trovo i pesi in {CUT_WEIGHTS_SOURCE}")



In [ ]:
print("\nAvvio Generazione (Real -> Synthetic)...")


!python test.py \
  --dataroot ./datasets/ego_adaptation \
  --name ego_cut_kaggle \
  --CUT_mode CUT \
  --phase test \
  --epoch latest \
  --num_test {num_img} \
  --direction AtoB \
  --results_dir {RESULTS_DIR} \
  --preprocess none \
  --no_dropout \
  --eval 

print(f"\nLe immagini generate sono in: {RESULTS_DIR}")

In [ ]:
# --- 6. CREAZIONE ARCHIVIO ZIP ---
import shutil
import os

ZIP_FILENAME = "/kaggle/working/dataset_generato_completo"

print("-" * 50)
print(f"Sto zippando la cartella: {RESULTS_DIR}")

try:
    shutil.make_archive(ZIP_FILENAME, 'zip', RESULTS_DIR)
    
    full_zip_path = ZIP_FILENAME + ".zip"
    if os.path.exists(full_zip_path):
        size_mb = os.path.getsize(full_zip_path) / (1024 * 1024)
        print(f"\nZIP CREATO CON SUCCESSO!")
        print(f"Percorso: {full_zip_path}")
        print(f"Dimensione: {size_mb:.2f} MB")
    else:
        print("\nErrore: Il file zip non è stato trovato dopo la creazione.")

except Exception as e:
    print(f"\nErrore durante la compressione: {e}")